In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import torch
import torchvision.transforms as transforms
from PIL import Image

In [3]:
!pip install nltk

In [4]:
from nltk.translate.bleu_score import sentence_bleu

In [5]:
#sentence = 'Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false'.split(",")

In [6]:
#s2 = [word.strip().lower() for word in sentence]

In [7]:
#s2

In [8]:
#candidate = 'head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false'.split(",")

In [9]:
#c2 = [cand.strip().lower() for cand in candidate]

In [10]:
#c2

In [11]:
#reference = [s2]
#candidate = 'head : b , anterior - scutum : y , inner - alar - space : b , scutellum : y , t1 : y , t2 : y / o , t3 : b , t4 : b , t5 : b , t6 : b , corbicular'.split(",")
#print(f'BLEU score: {sentence_bleu(reference, c2, weights=(1,0,0,0))}')

In [12]:
#print(f'BLEU score: {sentence_bleu(reference, c2, weights=(0.5,0.5,0,0))}')

In [13]:
#print(f'BLEU score: {sentence_bleu(reference, c2, weights=(0.33,0.33,0.33,0))}')

In [14]:
#print(f'BLEU score: {sentence_bleu(reference, c2, weights=(0.25,0.25,0.25,0.25))}')

In [15]:
#sentence_bleu([['Three', 'people', 'running', 'on', 'the', 'road']], ['Three', 'people', 'running', 'on', 'the', 'road'], weights=(1,0,0,0))

In [16]:
def print_examples(model, device, dataset):
    transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    model.eval()
    test_img1 = transform(Image.open("../../dataset_3/training/Bombus_affinis/Affinis_7.jpg").convert("RGB")).unsqueeze(
        0
    )
    print("Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false")
    print(
        "Example 1 Model Prediction: "
        + " ".join(model.caption_image(test_img1.to(device), dataset.vocab))
    )
    
    sentence = '<sos> Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false <eos>'.split(",")
    s2 = [word.strip().lower() for word in sentence]
    reference = [s2]
    #candidate = 'head : b , anterior - scutum : y , inner - alar - space : b , scutellum : y , t1 : y , t2 : y / o , t3 : b , t4 : b , t5 : b , t6 : b , corbicular'.split(",")
    pred = " ".join(model.caption_image(test_img1.to(device), dataset.vocab))
    candidate = pred.split(",")
    c2 = [cand.strip().lower() for cand in candidate]
    
    print(f'reference: {reference}')
    print(f'c2: {c2}')
    print(f'BLEU 1-gram score: {sentence_bleu(reference, c2, weights=(1,0,0,0))}')
    print(f'BLEU 2-gram score: {sentence_bleu(reference, c2, weights=(0.5,0.5,0,0))}')
    print(f'BLEU 3-gram score: {sentence_bleu(reference, c2, weights=(0.33,0.33,0.33,0))}')
    print(f'BLEU 4-gram score: {sentence_bleu(reference, c2, weights=(0.25,0.25,0.25,0.25))}')
    
    
#     test_img2 = transform(
#         Image.open("test_examples/child.jpg").convert("RGB")
#     ).unsqueeze(0)
#     print("Example 2 CORRECT: Child holding red frisbee outdoors")
#     print(
#         "Example 2 OUTPUT: "
#         + " ".join(model.caption_image(test_img2.to(device), dataset.vocab))
#     )
#     test_img3 = transform(Image.open("test_examples/bus.png").convert("RGB")).unsqueeze(
#         0
#     )
#     print("Example 3 CORRECT: Bus driving by parked cars")
#     print(
#         "Example 3 OUTPUT: "
#         + " ".join(model.caption_image(test_img3.to(device), dataset.vocab))
#     )
#     test_img4 = transform(
#         Image.open("test_examples/boat.png").convert("RGB")
#     ).unsqueeze(0)
#     print("Example 4 CORRECT: A small boat in the ocean")
#     print(
#         "Example 4 OUTPUT: "
#         + " ".join(model.caption_image(test_img4.to(device), dataset.vocab))
#     )
#     test_img5 = transform(
#         Image.open("test_examples/horse.png").convert("RGB")
#     ).unsqueeze(0)
#     print("Example 5 CORRECT: A cowboy riding a horse in the desert")
#     print(
#         "Example 5 OUTPUT: "
#         + " ".join(model.caption_image(test_img5.to(device), dataset.vocab))
#     )
    model.train()

In [17]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [18]:
def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    step = checkpoint["step"]
    return step